In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the working directory to the root of the project folder
root_path = '/content/drive/MyDrive/Projects/StartupValuation/'
data_path = root_path + 'Data/'

## Data Preparation

In [4]:
# ACQUISITIONS
# Read the acquisitions data
acquisitions = pd.read_csv(data_path + 'acquisitions.csv')

# Drop unnecessary columns
acquisitions = acquisitions.drop(columns=['id', 'acquisition_id', 'created_at', 'updated_at',
                                          'source_url', 'source_description'])

# Convert columns to appropriate types
acquisitions['term_code'] = acquisitions['term_code'].astype('category')
acquisitions['price_currency_code'] = acquisitions['price_currency_code'].astype('category')
acquisitions['acquired_at'] = pd.to_datetime(acquisitions['acquired_at'])

# Remove duplicate rows
acquisitions = acquisitions.drop_duplicates()

# # Display the first few rows of the DataFrame
# print(acquisitions.head())

In [5]:
# FUNDING_ROUNDS
funding_rounds = pd.read_csv(data_path + 'funding_rounds.csv')

funding_rounds = funding_rounds.drop(columns=['id', 'created_at', 'updated_at', 'created_by',
                                              'raised_amount', 'raised_currency_code', 'pre_money_valuation',
                                              'post_money_valuation_usd', 'pre_money_currency_code',
                                              'post_money_valuation', 'pre_money_valuation_usd',
                                              'post_money_currency_code', 'source_url'])
funding_rounds['funded_at'] = pd.to_datetime(funding_rounds['funded_at'])
funding_rounds['funding_round_type'] = funding_rounds['funding_round_type'].astype('category')
funding_rounds['funding_round_code'] = funding_rounds['funding_round_code'].astype('category')
funding_rounds['is_first_round'] = funding_rounds['is_first_round'].astype('category')
funding_rounds['is_last_round'] = funding_rounds['is_last_round'].astype('category')

funding_rounds.loc[(funding_rounds['funding_round_code'] == "angel") &
                   (funding_rounds['funding_round_type'] == "series-a"), 'funding_round_type'] = "angel"

funding_rounds.loc[(funding_rounds['funding_round_code'] == "seed") &
                   (funding_rounds['funding_round_type'] == "series-a"), 'funding_round_type'] = "angel"

In [6]:
# FUNDS
funds = pd.read_csv(data_path + 'funds.csv')

funds = funds.drop(columns=['id', 'created_at', 'updated_at', 'source_url', 'source_description'])
funds['funded_at'] = pd.to_datetime(funds['funded_at'])
funds['raised_currency_code'] = funds['raised_currency_code'].astype('category')
funds = funds.drop_duplicates()

In [7]:
# INVESTMENTS
investments = pd.read_csv(data_path + 'investments.csv')
investments = investments.drop(columns=['id', 'created_at', 'updated_at']).drop_duplicates()

In [8]:
# IPOS
ipos = pd.read_csv(data_path + 'ipos.csv')

ipos = ipos.drop(columns=['id', 'created_at', 'updated_at', 'stock_symbol', 'source_url', 'source_description'])
ipos['valuation_currency_code'] = ipos['valuation_currency_code'].astype('category')
ipos['raised_currency_code'] = ipos['raised_currency_code'].astype('category')
ipos['public_at'] = pd.to_datetime(ipos['public_at'])
ipos = ipos.drop_duplicates()

In [9]:
# MILESTONES
milestones = pd.read_csv(data_path + 'milestones.csv')

milestones = milestones.drop(columns=['id', 'created_at', 'updated_at', 'source_url', 'milestone_code'])
milestones['milestone_at'] = pd.to_datetime(milestones['milestone_at'])
milestones = milestones.drop_duplicates()

In [10]:
# OBJECTS
objects = pd.read_csv(data_path + 'objects.csv')

objects = objects.drop(columns=['normalized_name', 'permalink', 'created_at', 'updated_at',
                                'first_investment_at', 'last_investment_at', 'created_by', 'permalink',
                                'domain', 'twitter_username', 'logo_url', 'overview'])
objects['founded_at'] = pd.to_datetime(objects['founded_at'])
objects['logo_width'] = pd.to_numeric(objects['logo_width'])
objects['logo_height'] = pd.to_numeric(objects['logo_height'])
objects['country_code'] = objects['country_code'].astype('category')
objects['state_code'] = objects['state_code'].astype('category')
objects['investment_rounds'] = pd.to_numeric(objects['investment_rounds'])
objects['invested_companies'] = pd.to_numeric(objects['invested_companies'])
objects['first_funding_at'] = pd.to_datetime(objects['first_funding_at'])
objects['last_funding_at'] = pd.to_datetime(objects['last_funding_at'])
objects['funding_rounds'] = pd.to_numeric(objects['funding_rounds'])
objects['funding_total_usd'] = pd.to_numeric(objects['funding_total_usd'])
objects['first_milestone_at'] = pd.to_datetime(objects['first_milestone_at'])
objects['milestones'] = pd.to_numeric(objects['milestones'])
objects['relationships'] = pd.to_numeric(objects['relationships'])

<ipython-input-10-483ebc319f77>:2: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  objects = pd.read_csv(data_path + 'objects.csv')


In [11]:
# OFFICES
offices = pd.read_csv(data_path + 'offices.csv')

offices = offices.drop(columns=['id', 'zip_code', 'created_at', 'updated_at']).drop_duplicates()
offices['state_code'] = offices['state_code'].astype('category')

In [12]:
# PEOPLE
people = pd.read_csv(data_path + 'people.csv')

people = people.drop(columns=['id']).drop_duplicates()

In [13]:
# RELATIONSHIPS
relationships = pd.read_csv(data_path + 'relationships.csv')

relationships = relationships.drop(columns=['id', 'sequence', 'created_at', 'updated_at']).drop_duplicates()
relationships['is_past'] = relationships['is_past'].astype('category')

In [14]:
# Filter and select columns for STARTUPS
STARTUPS = objects[
    (objects['entity_type'] == 'Company') &
    (objects['status'] != '') &
    ~(objects['country_code'].isin(['CSS', 'FST']))
].drop('entity_id', axis=1).drop_duplicates()

# Filter and select columns for FINANCIAL_ORG
FINANCIAL_ORG = objects[
    objects['entity_type'] == 'FinancialOrg'
].drop(['closed_at', 'entity_id', 'parent_id', 'category_code', 'status', 'funding_rounds',
        'funding_total_usd', 'first_funding_at', 'last_funding_at', 'milestones',
        'last_milestone_at', 'first_milestone_at'], axis=1)

# Filter and select columns for Product
Product = objects[
    objects['entity_type'] == 'Product'
].drop(['region', 'entity_type', 'investment_rounds', 'invested_companies', 'funding_rounds',
        'first_funding_at', 'last_funding_at', 'funding_total_usd', 'relationships'], axis=1)

# Filter and select columns for Person
Person = objects[
    objects['entity_type'] == 'Person'
]

# Convert columns to appropriate data types (if needed)
categorical_columns = ['category_code', 'status', 'country_code']
STARTUPS[categorical_columns] = STARTUPS[categorical_columns].astype('category')


In [15]:
# Merge STARTUPS and ipos dataframes
dataset = pd.merge(STARTUPS, ipos, left_on="id", right_on="object_id", how="left")
dataset.drop(columns=['homepage_url', 'parent_id', 'entity_type', 'short_description', 'description',
                     'tag_list', 'valuation_amount', 'valuation_currency_code', 'raised_amount',
                     'public_at', 'raised_currency_code', 'ipo_id'], inplace=True)

# Merge with funding_rounds and filter for non-null funding_round_type
rounds = pd.merge(dataset, funding_rounds, left_on="id", right_on="object_id", how="left")
# print(rounds.head())
rounds = rounds[rounds['funding_round_type'].notna()]
rounds_grouped = rounds.groupby('id')['raised_amount_usd'].agg([
    ('angel', lambda x: x[rounds['funding_round_type'] == "angel"].sum()),
    ('crowdfunding', lambda x: x[rounds['funding_round_type'] == "crowdfunding"].sum()),
    ('other', lambda x: x[rounds['funding_round_type'] == "other"].sum()),
    ('post_ipo', lambda x: x[rounds['funding_round_type'] == "post-ipo"].sum()),
    ('private_equity', lambda x: x[rounds['funding_round_type'] == "private_equity"].sum()),
    ('series_a', lambda x: x[rounds['funding_round_type'] == "series-a"].sum()),
    ('series_b', lambda x: x[rounds['funding_round_type'] == "series-b"].sum()),
    ('series_c', lambda x: x[rounds['funding_round_type'] == "series-c+"].sum()),
    ('venture', lambda x: x[rounds['funding_round_type'] == "venture"].sum())
]).reset_index()

dataset = pd.merge(dataset, rounds_grouped, on="id", how="left")

# Number of acquisitions made
t = acquisitions.groupby('acquiring_object_id').size().reset_index(name='num_acquisizioni_effettuate')
dataset = pd.merge(dataset, t, left_on="id", right_on="acquiring_object_id", how="left")
dataset['num_acquisizioni_effettuate'].fillna(0, inplace=True)

# Have been acquired
t = acquisitions.assign(have_been_acquired=1).groupby('acquired_object_id')['have_been_acquired'].first().reset_index()
dataset = pd.merge(dataset, t, left_on="id", right_on="acquired_object_id", how="left")
dataset['have_been_acquired'].fillna(0, inplace=True)

# Financial organization financed
t = pd.merge(investments, FINANCIAL_ORG, left_on="investor_object_id", right_on="id")
t = t.groupby('funded_object_id').size().reset_index(name='n')
t['fin_org_financed'] = 1
dataset = pd.merge(dataset, t[['funded_object_id', 'fin_org_financed']], left_on="id", right_on="funded_object_id", how="left")
dataset['fin_org_financed'].fillna(0, inplace=True)

# Person financed
t = pd.merge(investments, Person, left_on="investor_object_id", right_on="id")
t = t.groupby('funded_object_id').size().reset_index(name='n')
t['person_financed'] = 1
dataset = pd.merge(dataset, t[['funded_object_id', 'person_financed']], left_on="id", right_on="funded_object_id", how="left")
dataset['person_financed'].fillna(0, inplace=True)

# Startup financed
t = pd.merge(investments, STARTUPS, left_on="investor_object_id", right_on="id")
t = t.groupby('funded_object_id').size().reset_index(name='n')
t['startup_financed'] = 1
dataset = pd.merge(dataset, t[['funded_object_id', 'startup_financed']], left_on="id", right_on="funded_object_id", how="left")
dataset['startup_financed'].fillna(0, inplace=True)

# Number of products
t = Product.groupby('parent_id').size().reset_index(name='num_prodotti')
t = t[t['parent_id'] != ""]
dataset = pd.merge(dataset, t, left_on="id", right_on="parent_id", how="left")
dataset['num_prodotti'].fillna(0, inplace=True)

# Drop and distinct
dataset.drop(columns=['first_milestone_at', 'last_milestone_at', 'last_funding_at',
                     'first_funding_at', 'name', 'city', 'region', 'closed_at', 'state_code'], inplace=True)
dataset.drop_duplicates(inplace=True)

# Formatting
dataset['category_code'] = pd.Categorical(dataset['category_code'])
dataset['status'] = pd.Categorical(dataset['status'])
dataset['country_code'] = pd.Categorical(dataset['country_code'])
dataset['fin_org_financed'] = pd.Categorical(dataset['fin_org_financed'])
dataset['person_financed'] = pd.Categorical(dataset['person_financed'])
dataset['startup_financed'] = pd.Categorical(dataset['startup_financed'])

In [16]:
# To display all column names
print(dataset.columns)
dataset.describe()

Index(['id', 'category_code', 'status', 'founded_at', 'logo_width',
       'logo_height', 'country_code', 'investment_rounds',
       'invested_companies', 'funding_rounds', 'funding_total_usd',
       'milestones', 'relationships', 'object_id', 'angel', 'crowdfunding',
       'other', 'post_ipo', 'private_equity', 'series_a', 'series_b',
       'series_c', 'venture', 'acquiring_object_id',
       'num_acquisizioni_effettuate', 'acquired_object_id',
       'have_been_acquired', 'funded_object_id_x', 'fin_org_financed',
       'funded_object_id_y', 'person_financed', 'funded_object_id',
       'startup_financed', 'parent_id', 'num_prodotti'],
      dtype='object')


,founded_at,logo_width,logo_height,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,angel,...,other,post_ipo,private_equity,series_a,series_b,series_c,venture,num_acquisizioni_effettuate,have_been_acquired,num_prodotti
count,91181,196498.000000,196498.000000,196498.000000,196498.000000,196498.000000,1.964980e+05,196498.000000,196498.000000,3.170300e+04,...,3.170300e+04,3.170300e+04,31703.0,3.170300e+04,3.170300e+04,3.170300e+04,3.170300e+04,196498.000000,196498.000000,196498.000000
mean,2005-12-21 06:41:29.018545536,257.161605,124.729819,0.031278,0.029074,0.267799,2.101753e+06,0.559568,1.881556,3.087273e+05,...,1.367331e+06,4.616268e+05,0.0,1.637946e+06,1.743996e+06,2.802982e+06,3.875335e+06,0.048438,0.047802,0.139126
min,1901-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,2004-02-10 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
50%,2009-01-01 00:00:00,143.000000,52.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
75%,2011-03-01 00:00:00,295.000000,120.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,2.000000,8.600000e+04,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,9.174550e+05,0.000000,0.000000,0.000000
max,2014-10-01 00:00:00,18200.000000,9600.000000,478.000000,459.000000,15.000000,5.700000e+09,9.000000,1189.000000,2.250000e+08,...,3.985050e+09,5.620000e+09,0.0,6.817591e+08,3.551870e+08,1.085000e+09,1.710000e+09,159.000000,1.000000,152.000000
std,NaN,500.263083,272.667095,1.423567,1.336926,0.778318,2.603847e+07,0.702945,7.514966,2.017510e+06,...,3.155952e+07,3.427218e+07,0.0,9.176194e+06,7.370781e+06,1.838624e+07,2.119789e+07,0.893161,0.213348,1.048590


## Data Upload

In [19]:
final_dataset_path = root_path + 'dataset.csv'
dataset.to_csv(final_dataset_path, index=False)
print(f"Processed data saved to {final_dataset_path}")

Processed data saved to /content/drive/MyDrive/Projects/StartupValuation/dataset.csv
